**Lib**

In [13]:
from google.colab import drive
import pandas as pd
import os
from tqdm import tqdm
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Concatenate, Resizing, Rescaling, RandomFlip, \
                        RandomRotation, Dropout, GlobalAveragePooling2D, RandomZoom, Flatten
from keras.models import Model, Input, Sequential
from tensorflow.keras.optimizers import Adam 
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import numpy as np

Drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


**Path**

In [3]:
model_path = '/content/drive/MyDrive/Projects/IS254/Code/models'
checkpoint_path = '/content/drive/MyDrive/Projects/IS254/Code/checkpoints'
data_path = '/content/drive/MyDrive/Projects/IS254/Code/image_by_class'

**Const**

In [4]:
img_size_target = 224
batch_size = 32
num_classes = 3
epochs = 60
seed = 111

**Load Data**

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_path + '/train',
  seed=seed,
  color_mode = 'rgb',
  image_size=(img_size_target, img_size_target), # resize
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_path + '/valid',
  seed=seed,
  color_mode = 'rgb',
  image_size=(img_size_target, img_size_target), # resize
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  data_path + '/test',
  seed=seed,
  color_mode = 'rgb',
  image_size=(img_size_target, img_size_target), # resize
  batch_size=batch_size)

Found 1182 files belonging to 3 classes.
Found 150 files belonging to 3 classes.
Found 146 files belonging to 3 classes.


**Stage 1**

In [6]:
AUTOTUNE = tf.data.AUTOTUNE

enhance = Sequential([
    Rescaling(1./255)  # rescaling
    ], name='enhance')

augmentation = Sequential([
    RandomFlip(seed=seed, mode='horizontal'), # flip
   ], name='augmentation')

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x, y: (enhance(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

**Load Models**

final 

In [7]:
os.listdir(model_path)

['vgg16-final.h5', 'vgg16_midnight-final.h5']

In [8]:
model_vgg16 = tf.keras.models.load_model(model_path + '/' + 'vgg16_midnight-final.h5')

best checkpoint

In [9]:
os.listdir(checkpoint_path)

['vgg16-001-0.407122-0.383459.h5',
 'vgg16-002-0.469682-0.578947.h5',
 'vgg16-003-0.521655-0.601504.h5',
 'vgg16-004-0.504331-0.541353.h5',
 'vgg16-005-0.529355-0.593985.h5',
 'vgg16-006-0.544755-0.624060.h5',
 'vgg16-008-0.546679-0.601504.h5',
 'vgg16-009-0.528393-0.616541.h5',
 'vgg16-011-0.548604-0.639098.h5',
 'vgg16-016-0.559192-0.631579.h5',
 'vgg16-017-0.561116-0.624060.h5',
 'vgg16-021-0.550529-0.669173.h5',
 'vgg16-022-0.572666-0.661654.h5',
 'vgg16-024-0.566891-0.646617.h5',
 'vgg16-025-0.574591-0.646617.h5',
 'vgg16-032-0.589990-0.654135.h5',
 'vgg16-035-0.564966-0.646617.h5',
 'vgg16-036-0.567854-0.661654.h5',
 'vgg16-044-0.581328-0.654135.h5',
 'vgg16-050-0.573628-0.661654.h5',
 'vgg16-051-0.574591-0.661654.h5',
 'vgg16_midnight-001-0.398477-0.520000.h5',
 'vgg16_midnight-002-0.494924-0.580000.h5',
 'vgg16_midnight-003-0.509306-0.526667.h5',
 'vgg16_midnight-004-0.489848-0.480000.h5',
 'vgg16_midnight-005-0.526227-0.566667.h5',
 'vgg16_midnight-006-0.540609-0.586667.h5',
 

In [11]:
best_vgg16 = tf.keras.models.load_model(checkpoint_path + '/' + 'vgg16_midnight-029-0.604061-0.666667.h5')

**Evaluate**

final model

In [14]:
y_pred_total = []
y_true = []

for img, label in test_ds:
    y_pred = model_vgg16.predict(img)
    y_pred_total += np.argmax(y_pred, axis=-1).tolist() 
    y_true += np.array(label).flatten().tolist()

print('Accuracy: ', accuracy_score(y_true, y_pred_total)*100)
print('F1-score: ',f1_score(y_true, y_pred_total, average='macro')*100)
cm = confusion_matrix(y_true, y_pred_total)
plt.show()
cm

Accuracy:  62.328767123287676
F1-score:  62.410184317400805


array([[39,  0,  8],
       [ 4, 28, 17],
       [ 7, 19, 24]])

best checkpoint

In [16]:
y_pred_total = []
y_true = []
for img, label in test_ds:
    y_pred = best_vgg16.predict(img)
    y_pred_total += np.argmax(y_pred, axis=-1).tolist() 
    y_true += np.array(label).flatten().tolist()

print('Accuracy: ', accuracy_score(y_true, y_pred_total)*100)
print('F1-score: ',f1_score(y_true, y_pred_total, average='macro')*100)
cm = confusion_matrix(y_true, y_pred_total)
cm

Accuracy:  64.38356164383562
F1-score:  63.858005358944915


array([[41,  0,  6],
       [ 5, 30, 14],
       [10, 17, 23]])